In [1]:
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from pydub import AudioSegment
import glob
import pandas as pd

Using TensorFlow backend.


In [2]:
df = pd.DataFrame()

df['path'] = glob.glob("../outputs/splited_audio/**.mp3")

df['id_person'] = df.path.apply(lambda x: (x.split("/")[3]).split('_')[0])
df['id_audio'] = df.path.apply(lambda x: ((x.split("/")[3]).split('_')[1]).split('.')[0])
df['array'] = df.path.apply(lambda x: (list(AudioSegment.from_file(x, format='mp3').get_array_of_samples())))


In [3]:
df.head()

,path,id_person,id_audio,array
0,../outputs/splited_audio/2414_4.mp3,2414,4,"[14, 171, 155, -18, -94, -38, -87, -231, -209,..."
1,../outputs/splited_audio/3080_4.mp3,3080,4,"[-957, -1389, -1545, -1610, -1693, -1791, -177..."
2,../outputs/splited_audio/2033_10.mp3,2033,10,"[3, 3, 2, 2, 2, 3, 1, 1, 1, 0, -1, 0, 0, 1, 2,..."
3,../outputs/splited_audio/3997_9.mp3,3997,9,"[48, -40, -60, 50, 17, -92, 27, 173, 3, -211, ..."
4,../outputs/splited_audio/3764_12.mp3,3764,12,"[110, 380, 570, 515, 154, -473, -978, -922, -4..."


In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

y = df['id_person'].to_numpy()
X = np.vstack(df['array'])

print(y.shape, X.shape)


(899,) (899, 320000)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(719, 320000) (180, 320000) (719,) (180,)


In [19]:
# https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
# https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

input_dim, timesteps = X_train.shape
print(f"Input dimensions:{input_dim} Timesteps:{timesteps}")

from keras.models import Sequential

model = Sequential()
model.add(LSTM(50, input_shape=(timesteps//50,50)))
model.add(LSTM(30,return_sequences=True))



model.compile(optimizer='adam', loss='mse')
model.summary()

Input dimensions:719 Timesteps:320000


ValueError: Input 0 is incompatible with layer lstm_7: expected ndim=3, found ndim=2

In [16]:
#XTr = X_train.reshape((X_train.shape[0],8000,40))
#yTr = y_train
#XT = X_test.reshape((X_test.shape[0],8000,40))
#yT = y_test



XTr = np.expand_dims(X_train,axis=2).reshape((X_train.shape[0],X_train.shape[1]//50,50))
yTr = y_train
XT = np.expand_dims(X_test,axis=2).reshape((X_test.shape[0],X_train.shape[1]//50,50))
yT = y_test

print(XTr.shape,yTr.shape,XT.shape,yT.shape)

(719, 6400, 50) (719,) (180, 6400, 50) (180,)


In [17]:
model.fit(XTr, yTr,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(XT, yT))

ValueError: Error when checking target: expected lstm_3 to have shape (50,) but got array with shape (1,)